In [1]:
import sys

sys.path.append("../")

In [2]:
import spacy
from src.loader import TextLoader
from src.purifier import Purifier
from tqdm import tqdm


/home/marek/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
spacy.prefer_gpu()

True

In [4]:
nlp = spacy.load("en_core_web_lg")

In [5]:
TextLoader.DATASET_PATH = "../data/dataset"
TextLoader.TRAIN_DIR = TextLoader.DATASET_PATH + "/train"
TextLoader.TEST_DIR = TextLoader.DATASET_PATH + "/test"
TextLoader.TRAIN_CSV = TextLoader.DATASET_PATH + "/train.csv"
loader = TextLoader()

In [6]:
text = loader.load_text_with_id("6A2F708CAA8B")
purified = Purifier().purify(text.text)

In [7]:
print(repr(text))

"Texting and driving\n\nThe use of phones and cars serve many purposes such as communicating with others, playing games, and using the many different internet based features.\n\nWe use cars as a means of travel, others to modify and race, and make them look cool. In every day jobs like firefighters and police use cars to patrol neighborhoods and to travel to the location they are called to. For delivery like packages and food we use cars and phones for most of the things we do in modern society. But get in some accidents or cause accidents with cars and we get hurt because of these accidents physically and financially but they can be avoided taking precautions like insurance and driving safe and keep your car maintenance high or good . According to https://www. insurancejournal. com/news/national/2018/08/24/499107. htm\n\nThe death toll for 2018\n\nis 18,720 and an additional 2.1 million people who have sustained serious injuries due to car accidents in the first six months of 2018. Te

In [8]:
print(repr(purified))

"Texting and driving The use of phones and cars serve many purposes such as communicating with others, playing games, and using the many different internet based features. We use cars as a means of travel, others to modify and race, and make them look cool. In every day jobs like firefighters and police use cars to patrol neighborhoods and to travel to the location they are called to. For delivery like packages and food we use cars and phones for most of the things we do in modern society. But get in some accidents or cause accidents with cars and we get hurt because of these accidents physically and financially but they can be avoided taking precautions like insurance and driving safe and keep your car maintenance high or good. According to https://www. insurancejournal. com/news/national/2018/08/24/499107. htm The death toll for 2018 is 18,720 and an additional 2.1 million people who have sustained serious injuries due to car accidents in the first six months of 2018. Texting and dri

In [9]:
texts = []
for text in loader:
    text = loader.load_text_with_id(text, True)
    texts.append(text)
    
    if len(texts) == 1000:
        break


In [10]:
sum([len(text.text) for text in texts])

2529035

In [11]:
first_text = texts[0]
purified = Purifier().purify(first_text.text)

token = nlp(purified)[0]
print(token.vector.shape, token.vector_norm)
print(token.vector)

(300,) 7.319231
[-0.23002   -0.17722    0.1987     0.074753   0.43008   -0.32335
  0.51367   -0.93191   -0.13008    1.1388     0.24475   -0.11143
  0.414     -0.49195   -0.35074   -0.17618   -0.49492    1.6227
  0.13684    0.20892   -0.28064    0.91007    0.18892   -0.084036
  0.059605  -0.4257    -0.80634   -0.5988     0.23514    0.25941
  0.37393   -0.4636    -0.18336    0.22496    0.3944     0.31794
  0.0077045 -0.02932    0.18534    0.52041   -0.40609   -0.081746
  0.053663   0.44023   -0.61921    0.42234   -0.43615   -0.19039
 -0.01895    0.19229   -0.04482   -0.39512   -0.44173   -0.21613
  0.16226   -0.15322   -0.2715    -0.43603    0.11674   -0.067926
 -0.031838  -0.14305    0.54115    0.54081    0.14381   -0.051979
 -0.022535   0.33687   -0.34956   -0.27459   -0.15953    0.50201
  0.22368    0.18312    0.42675   -0.076709   0.76353   -0.50104
  0.8809     1.3147     0.12443    0.029498  -0.51311    0.43192
  0.088301   0.28055   -0.21525    0.44005    0.57639   -0.28996
  0.02

In [12]:
# Out of vocabulary - does not have embedding vector
oov_tokens = {}
for text in tqdm(texts):
    purified_text = Purifier.purify(text.text)
    for token in nlp(purified_text):
        if token.is_oov:
            if text.text_id not in oov_tokens:
                oov_tokens[text.text_id] = [token]
            else:
                oov_tokens[text.text_id].append(token)

100%|██████████| 1000/1000 [00:29<00:00, 33.38it/s]


In [13]:
len(oov_tokens)

583

In [14]:
for text_id, tokens in oov_tokens.items():
    print(text_id)
    for token in tokens:
        print(repr(token.text))
    print("\n----\n")

50B3435E475B
'3,328'

----

A97DE0D49AEA
'DrivingHundreds'
'Crundall'
'NCSL'
'Crundall'
'NCSL'
'concertation'
'Crundall'
'Crundall'
'..'
'https://theconversation'
'free-71833'
'NCSL'
'ncsl'

----

48D3F4243F0F
'mischances'

----

4F0E197053FF
'accadences'

----

86C1ED49C35F
'ususage'

----

019328A0D7A3
'"The'

----

20FD63F49519
'impatiences'
'Occuping'

----

D0CBBD43827C
'3,477'

----

62644C50869C
'"Also'
'Legalmatch'
'process).Texting'
'edgarsnyder'
'cars/2018/02/28'
'fatalities/376802002/'
'legalmatch'

----

0421128DEE6C
'3,166'
'15,341'

----

7FF6281EC288
'druving'

----

89808E74DDC5
'AT&T'
'itcanwait'
'32,083,847'
'itcanwait'

----

2F2607C7D8F7
'insatiately'

----

4B30291A725D
'Teensafe'
'Teenlife'

----

E527586F851C
'32,999'
'2,239,000'
'1,2018'

----

5F1CF4B91975
'8could'

----

D14A82EE41BF
'netsafeutah'
'staysafe'
'distracted_driving'
'articles/2011/10/06/3333955'

----

AA994A6CAF65
'390,00'

----

6A2F708CAA8B
'insurancejournal'
'national/2018/08/24/499107'

----
